In [1]:
import Trough_GUI

['Checking Motor Calibration. Please wait...']
['Trough ready']


In [2]:
Trough_GUI.Monitor_Calibrate.Monitor_Setup_Trough(Trough_GUI.calibrations)

Starting step: 0 of close
tempindex_start: 0 tempindex_end: 84
[0.9658203650853481, 0.9653509234828496, 0.9634833880781865, 0.9621863521134048, 0.9606429379139518, 0.9585831134564644, 0.9575682515750363, 0.9562193742932529, 0.9546965699208444, 0.9532800455274458] [2.2706272943612372e-05, 3.8023430721201664e-05, 4.3520862521290416e-05, 4.29219553769953e-05, 4.096474087206654e-05, 7.608141694974527e-05, 3.070265820045002e-05, 4.8509376842192426e-05, 3.0853195521569764e-05, 4.6590893693080605e-05]
tempindex_start: 1 tempindex_end: 83
[0.9658203650853481, 0.9653509234828496, 0.9634833880781865, 0.9621863521134048, 0.9606429379139518, 0.9585831134564644, 0.9575682515750363, 0.9562193742932529, 0.9546965699208444, 0.9532800455274458] [2.2706272943612372e-05, 3.8023430721201664e-05, 4.3520862521290416e-05, 4.29219553769953e-05, 4.096474087206654e-05, 7.608141694974527e-05, 3.070265820045002e-05, 4.8509376842192426e-05, 3.0853195521569764e-05, 4.6590893693080605e-05]
tempindex_start: 2 tempind

Starting step: 6 of close
tempindex_start: 0 tempindex_end: 30
[0.4230574551720424, 0.426261058513115, 0.42743007915567277, 0.42975616413429163, 0.43162457595175263, 0.43476926390609005, 0.4379656992084431, 0.440042216358839, 0.44214511873350915, 0.44661996178691654] [8.053704295136431e-05, 4.694002264871925e-05, 6.282899822394212e-05, 6.49919994970594e-05, 7.528813176408325e-05, 8.125018707903217e-05, 4.63626083678876e-05, 6.264783003186338e-05, 7.699102556196435e-05, 9.392294366825576e-05]
tempindex_start: 1 tempindex_end: 29
[0.4230574551720424, 0.426261058513115, 0.42743007915567277, 0.42975616413429163, 0.43162457595175263, 0.43476926390609005, 0.4379656992084431, 0.440042216358839, 0.44214511873350915, 0.44661996178691654] [8.053704295136431e-05, 4.694002264871925e-05, 6.282899822394212e-05, 6.49919994970594e-05, 7.528813176408325e-05, 8.125018707903217e-05, 4.63626083678876e-05, 6.264783003186338e-05, 7.699102556196435e-05, 9.392294366825576e-05]
tempindex_start: 2 tempindex_end

Starting step: 2 of open
tempindex_start: 0 tempindex_end: 42
[0.13259102902374653, 0.13323730547628032, 0.13403166226912921, 0.1345322408924685, 0.13512338972528348, 0.13594459102902356, 0.13734828496042217, 0.13833773087071233, 0.13925329815303433, 0.14013381078024878] [3.575094067509739e-05, 4.426745088269927e-05, 2.990853286826422e-05, 3.102753250362805e-05, 3.059852673727748e-05, 4.98932960026648e-05, 6.520483939693177e-05, 4.662388570242474e-05, 3.9570656107412163e-05, 3.3569828709027016e-05]
tempindex_start: 1 tempindex_end: 41
[0.13259102902374653, 0.13323730547628032, 0.13403166226912921, 0.1345322408924685, 0.13512338972528348, 0.13594459102902356, 0.13734828496042217, 0.13833773087071233, 0.13925329815303433, 0.14013381078024878] [3.575094067509739e-05, 4.426745088269927e-05, 2.990853286826422e-05, 3.102753250362805e-05, 3.059852673727748e-05, 4.98932960026648e-05, 6.520483939693177e-05, 4.662388570242474e-05, 3.9570656107412163e-05, 3.3569828709027016e-05]
tempindex_start: 

tempindex_start: 0 tempindex_end: 30
[0.6693509234828496, 0.6748073878627968, 0.6815857519788918, 0.6868847238649805, 0.6929313984168866, 0.6993568601583113, 0.7032744063324539, 0.7033166226912928, 0.7033535620052771, 0.7033754890364844] [0.00018055150075905433, 0.0001495882612519352, 0.00020186412222875777, 0.00017028604326760853, 7.762323995177778e-05, 0.00019129307763723948, 2.1551138121254493e-05, 2.413835302915217e-05, 2.135244483719289e-05, 2.001166042775556e-05]
tempindex_start: 1 tempindex_end: 29
[0.6693509234828496, 0.6748073878627968, 0.6815857519788918, 0.6868847238649805, 0.6929313984168866, 0.6993568601583113, 0.7032744063324539, 0.7033166226912928, 0.7033535620052771, 0.7033754890364844] [0.00018055150075905433, 0.0001495882612519352, 0.00020186412222875777, 0.00017028604326760853, 7.762323995177778e-05, 0.00019129307763723948, 2.1551138121254493e-05, 2.413835302915217e-05, 2.135244483719289e-05, 2.001166042775556e-05]
tempindex_start: 2 tempindex_end: 28
[0.669350923482

In [ ]:
Trough_GUI.status_widgets.update_status({'bal_raw':1.27,'bal_dev':0.02,
                                         'barr_raw':0.8,'barr_dev':0.002,
                                         'temp_raw':3.30,'temp_dev':0.01,
                                         'messages':['No updates',]}, cals)

In [ ]:
cals.barriers.cal_apply([0.7613993559219399, 0.7613712807244503, 0.7614248322753392],
                       [2.684617327574853e-05, 2.057942843404166e-05, 2.3285347508477762e-05])

In [ ]:
from multiprocessing import Process
isinstance(TROUGH,Process)

In [ ]:
TROUGH.is_alive()

In [ ]:
import Trough_Control
if not Trough_Control.trough_util.is_trough_initialized():
    cmdsend, datarcv, TROUGH = Trough_Control.trough_util.init_trough()

In [ ]:
# Asynchronous update of status widgets
# TODO should this include the calibrations or simply be an asynchronous GUI
import threading
import Trough_GUI
trough_lock = threading.Lock()

def status_updater(trough_lock, cmdsend, datarcv, cals):
    import time
    datapkg = []
    run = True
    update = 0
    while run:
        min_next_time = time.time() + 2.0
        trough_lock.acquire()
        cmdsend.send(['Send',''])
        waiting = True
        while waiting:
            if datarcv.poll():
                datapkg =datarcv.recv()
                update_dict = {'barr_raw':datapkg[1][-1],
                               'barr_dev':datapkg[2][-1],
                               'bal_raw':datapkg[3][-1],
                               'bal_dev':datapkg[4][-1],
                               'temp_raw':datapkg[5][-1],
                               'temp_dev':datapkg[6][-1],
                               'messages':datapkg[7]}
                Trough_GUI.status_widgets.update_status(update_dict, cals)
                waiting = False
        trough_lock.release()
        if time.time()< min_next_time:
            time.sleep(min_next_time - time.time())
    return

status_update_thread = threading.Thread(target=status_updater, args=(trough_lock, cmdsend, datarcv, Trough_GUI.calibrations,))
status_update_thread.start()

In [ ]:
trough_lock.acquire()

In [ ]:
trough_lock.release()

In [ ]:
# Clean shutdown
cmdsend.send(['ShutDown',''])

In [ ]:
thread.is_alive()

In [ ]:
(5.1240567770745726e-05**2 + 3.623344097262064e-05**2)**0.5

In [ ]:
0.952894297560713 -0.951041761441179

In [ ]:
0.9557519788918206 -0.9546356109544173

In [ ]:
(3.3923859781379596e-05**2 + 4.5595401094938295e-05**2)**0.5

In [3]:
Trough_GUI.Monitor_Calibrate.pos_x

[1.0020219112079845,
 0.795646437994723,
 0.5949772127608539,
 0.39566562724874066,
 0.19663712849106374,
 -0.004856722096091337,
 0.10833725970599328,
 0.30587730870712393,
 0.5012274865205919,
 0.703537460882371]

In [8]:
Trough_GUI.Monitor_Calibrate.pos_y

[12.3, 10.5, 8.5, 6.35, 4.15, 2.45, 3.22, 5.3, 7.49, 9.52]

In [4]:
Trough_GUI.Monitor_Calibrate.open_speed_x

[1.0, 0.2, 0.9, 0.3, 0.8, 0.4, 0.7, 0.5, 0.6]

In [5]:
Trough_GUI.Monitor_Calibrate.open_speed_y

[0.00025876119497240377,
 2.6347990760394926e-05,
 0.00024148295822328948,
 5.7037935167317746e-05,
 0.00020702185186106703,
 8.859394218433566e-05,
 0.0001932872385289205,
 0.00014096479377809603,
 0.00020481168496205488]

In [6]:
Trough_GUI.Monitor_Calibrate.close_speed_y

[-1.926705661971805e-05,
 -0.00022648933615413937,
 -5.7029420665356145e-05,
 -0.00017720254787841663,
 -6.884415114322171e-05,
 -0.0001435124076901888,
 5.740124596753634e-05,
 -0.0001150041548637008,
 -9.932373475431898e-05,
 -0.00012311756598613538]

In [7]:
Trough_GUI.Monitor_Calibrate.close_speed_x

[0.1, 1.0, 0.2, 0.9, 0.3, 0.8, 0.4, 0.7, 0.5, 0.6]

ModuleNotFoundError: No module named 'jupyter_pandas_GUI'